
## In this notebook, you will see all the steps sequentially performed to be able to utilize the complete functionality of OAB framework. The steps are as follows :
0. SETUP
1. DATA
2. DATA SELECTION
3. PREPROCESSING
4. SAMPLING
5. ALGORITHM TRAINING AND TESTING
6. EVALUATION
7. SHOW BENCHMARK RESULTS
8. REPRODUCIBILTY
9. EXTENDING THE BENCHMARK(with own Algorithm)

This notebook focuses on <b>Unsupervised Image Data</b>. Let's begin!

# **0. SETUP**

`oab` framework can be integrated in your Python environment  as a `PyPi package`  using the following command:

In [ ]:
#ID 1(0)

#%%capture
# pip install oab
!pip install example-pkg-jd-kiel --extra-index-url=https://test.pypi.org/simple/

`Cloning` the repository:

`oab` is an open-source framework which can be accessed at https://github.com/ISDM-CAU-Kiel/oab. To use this .ipynb notebook successfully, the formerly mentioned repository needs to be cloned with the following command and this notebook must be run(if this is not the case already) within the cloned repository from the path:

<b>/oab/notebooks/benchmark_image/Unsupervised_Anomaly_Detection_on_Benchmark_Image_Data_v0.2.ipynb</b>

In [ ]:
#ID 2(0)
!git clone https://github.com/ISDM-CAU-Kiel/oab.git

Now, importing the necessary functions and internal variables :

In [2]:
#ID 3(0)

import sys
import os
from datetime import datetime 
from pathlib import Path         
sys.path.insert(0,f"{Path(os.getcwd()).parent.parent}")  
%load_ext autoreload
%autoreload 2

# necessary imports for loading datasets as well as information from recipe files
from oab.data.unsupervised import UnsupervisedAnomalyDataset
from oab.algorithms.unsupervised_wrapper import UnsupervisedWrapperToRecipe
from oab.data.load_image_dataset import _load_image_dataset
from oab.data.load_dataset import load_dataset
from oab.data.utils_image import image_datasets
from oab.data.utils import _append_to_yaml
from oab.data.load_recipe_functions import *

# necessary imports for algorithm comparisons and defining seeds
from oab.evaluation import EvaluationObject, ComparisonObject,all_metrics
import tensorflow as tf
import numpy as np
import random
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **0.1 NOTEBOOK AND CELL STRUCTURE** 

In this notebook there are certain sections where the user is required to enter its own information which are marked as comments of the form :

<b>### ADD YOUR CODE ###</b>  , so <b>###</b> can be searched to know what are those sections.

All cells are assigned an ID, as a comment at the top of the cell,for example as: <b>#ID 10(5)</b>, where 10 denotes the cell ID and 5 denotes the Section.

## 0.2 DETAILS OF THIS BENCHMARK RUN

In [3]:
#ID 4(0)


#timestamp is set  for this run  

time=datetime.now().strftime("%Y%m%d%H%M%S")

### ADD YOUR OWN BENCHMARK NAME ###
benchmark_name="Paper_A"
benchmark_type="usi"      #(usi denotes unsupervised image benchmark)
dataset_folder="datasets"
#creating folder for this benchmark with current timestamp name to store recipes of this run
if not os.path.exists(f"{benchmark_name}/{time}-{benchmark_type}"): #creating directory for this benchhmark run for storing recipes
    os.makedirs(f"{benchmark_name}/{time}-{benchmark_type}")
    
new_recipe_path=f"{benchmark_name}/{time}-{benchmark_type}/{time}-{benchmark_name}" 
print(f"{benchmark_name}/{time}-{benchmark_type}/{time}-{benchmark_name}")

Paper_A/20211223120704-usi/20211223120704-Paper_A


### For reproducing  previously-created recipes of another benchmark run, without adding new datasets and algorithms from this benchmark  ,  skip to :

### `#ID 20(5)`

# **1. DATA**

First of all, we will have a look at the Datasets that are pre-installed in OAB which can be used for benchmarking

In [4]:
#ID 5(1)
lst_all_datasetnames =image_datasets
for i,dataset in enumerate(lst_all_datasetnames):
    print(f"{i}.{dataset}")

0.mnist
1.fashion_mnist
2.cifar10
3.cifar100
4.mvtec_ad_carpet
5.mvtec_ad_grid
6.mvtec_ad_leather
7.mvtec_ad_tile
8.mvtec_ad_wood
9.mvtec_ad_bottle
10.mvtec_ad_cable
11.mvtec_ad_capsule
12.mvtec_ad_hazelnut
13.mvtec_ad_metal_nut
14.mvtec_ad_pill
15.mvtec_ad_screw
16.mvtec_ad_toothbrush
17.mvtec_ad_transistor
18.mvtec_ad_zipper
19.crack



`oab` provides a variety of image datasets that can easily be loaded, in either of the following ways: 

`1.` If a user is interested in using her own image dataset  loading **via local folder directory**, the following steps have to be followed: (1) Ensure that the format is readable by oab. This requires there to be a folder for the dataset with the subfolders `normal` and `anomaly`. Naturally, all normal images are in the folder `normal` and all images of anomalies in the folder `anomaly`. (2) Based on this folder structure, the dataset can be loaded.


"**Local folder structure - without URL usage**" :
```
dataset_name
        │
        ├── normal
        │    
        └── anomaly
``` 

`2.` If user's dataset is provided **via a URL**, then it would be downloaded and stored in the OAB's "datasets" folder, given that it is already formatted as per the required folder heirarchy, where the folder :

`good`(which should not be renamed) : contains normal images whereas 

`anomaly_folder_n`(can be renamed):  contains anomalous images



"**Uploaded folder structure - with URL usage**" :
```
dataset_name
        │
        ├── train
        │   ├── good
        │
        │  
        └── test
            ├── good
            ├── anomaly_folder_1
            ├── ...
            ├── ...
            ├── anomaly_folder_2
```
 
Note: Alternatively, it is of course possible to load the images into `numpy` arrays and treat them as if they were tabular data. If this approach is to be followed, please look at the notebooks for tabular data.


- Notes: Limited to 256x256

In [5]:
#ID 6(1)


#### ADD YOUR DATASETNAME(S) HERE ###

own_datasets_list=["myImageDataset"]  # More of user's own datasets can be added in this list
benchmark_datasets_list=['mnist']   # More of OAB's datasets can be added to this list


# 'myImageDataset' is the name of the Dataset(which the user loads for benchmarking as well as the name of the folder containing normal and anomaly folders
#  which further contain the respective images
#  and make sure folder structure is correct ({dataset_folder}/{name-without mvtec_ad_}/[normal/anomaly])




# calling the helper function to update internal OAB variables 
mvtec_ad_own_datasets_list=[]
for dataset_name in own_datasets_list:
 
  mvtec_ad_own_datasets_list.append(add_own_dataset(dataset_name))
print( f" Dataset(s) successfully added as :  {mvtec_ad_own_datasets_list}")




#Now,  we'll have a look at all the datasets again which are pre-installed in OAB,after adding own_datasets 
lst_all_datasetnames =image_datasets
print("Datasets of OAB:")
for i,dataset in enumerate(lst_all_datasetnames):
    print(f"{i}.{dataset}")

 Dataset(s) successfully added as :  ['mvtec_ad_myImageDataset']
Datasets of OAB:
0.mnist
1.fashion_mnist
2.cifar10
3.cifar100
4.mvtec_ad_carpet
5.mvtec_ad_grid
6.mvtec_ad_leather
7.mvtec_ad_tile
8.mvtec_ad_wood
9.mvtec_ad_bottle
10.mvtec_ad_cable
11.mvtec_ad_capsule
12.mvtec_ad_hazelnut
13.mvtec_ad_metal_nut
14.mvtec_ad_pill
15.mvtec_ad_screw
16.mvtec_ad_toothbrush
17.mvtec_ad_transistor
18.mvtec_ad_zipper
19.crack
20.mvtec_ad_myImageDataset


Now, If this dataset(s) was already stored in the dataset_folder, structured as mentioned in the initial description of this section above, then we have to create the file "applied_modification.txt" in Path(dataset_folder)/dataset_name/ "applied_modification.txt"). If this is not available in the location, then we download the dataset from the given URL(When data is downloaded for the URL, the orientation of the folders as well as the image resizing operation is performed and  information about that is stored in "applied_modification.txt")

In [6]:
#ID 7(1)
for dataset_name in own_datasets_list:
   open(Path(dataset_folder)/dataset_name/"applied_modification.txt", "w") 
   
  

# **2. DATA SELECTION**


Datasets can either be loaded directly as anomaly datasets or as classification datasets. In the former case, the dataset is automatically fully prepared and ready for sampling. In the latter case, further preprocessing is still possible and necessary.

Note that the automatic preprocessing for image datasets is to scale each value by `1/255`.

**After adding own dataset(s) in #ID 6(1),the user is able to load own dataset(s) using this method :**

In [7]:
#ID 8(2)


datasets={}  # contain dataset objects of own dataset names 


for dataset_name in mvtec_ad_own_datasets_list:  # loading own datasets 
    
    datasets[dataset_name[9:]]=_load_image_dataset(dataset_name ,anomaly_dataset=False,preprocess_classification_dataset=False,dataset_folder=dataset_folder)

### **2.1 Load anomaly detection datasets (with or without further preprocessing)**

In this section, we load some pre-installed data sets. This can be achieved using the `load_dataset` function. By default, it creates an anomaly dataset from which sampling is directly possible  but we can first create classifcation dataset and then anomaly dataset,either with the preprocessing applied (`preprocess_classification_dataset=True`) i.e. Scaling is performed on all values by the factor of 1/255, or without (`preprocess_classification_dataset=False`, default).

`In our case` we set have already imported own datasets with `anomaly_dataset=False ` and `preprocess_classification_dataset=False` in <b>#ID 8(2)</b> and we will also load the OAB datasets in the same way in <b>#ID 9(2)</b>


Note that as discussed in the paper, multiclass classification datasets like Cifar10 and MNIST are loaded with the class label `0` as normal label and all other labels as anomaly labels by default. (Alternatively, `oab` can automatically iterate through all classes as normal classes. This is not covered here.)

In [8]:
#ID 9(2)

#### ADD YOUR OWN NUMBER OF DATASETS AND FROM OAB FOR BENCHMARKING  ###

for dataset_name in benchmark_datasets_list:  # loading benchmark's datasets
    datasets[dataset_name]=load_dataset(dataset_name,anomaly_dataset=False,preprocess_classification_dataset=False,dataset_folder=dataset_folder)


In [9]:
#ID 10(2)
print(datasets)

{'myImageDataset': <oab.data.classification_dataset.ClassificationDataset object at 0x7f8fd4eeb510>, 'mnist': <oab.data.classification_dataset.ClassificationDataset object at 0x7f8fbbc54950>}


# **3. PREPROCESSING**

The  resizing of images(only when dataset is downloaded using URL) and scaling of images has already been performed while loading the datasets as shown in previous the Sections.

Standard preprocessing steps like deleting columns, encoding categorical values differently, or removing missing values do not apply to image data. Therefore, these methods (as well as own preprocessing steps and how these are captured) are covered in the tabular dataset benchmarks.

Here, we only show two preprocessing steps that are applied to datasets stored in `datasets` dictionary(loaded in 2.2), which can also be performed individually depending upon requirement :
- `Scale` all values by `1/255`.
- `Transform the dataset into an anomaly dataset` for unsupervised anomaly detection by setting the class label `0` to normal and all other class labels to anomalous.

In [12]:
#ID 11(3)                            SCALING APPLIED

                                         
for dataset_name in datasets:
    
    datasets[dataset_name].scale(scaling_factor=1/255)
    operations=datasets[dataset_name].operations_performed
    #NEW RECIPE CREATED
    datasets[dataset_name].write_operations_to_yaml(f"{new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml")
#print("Scaling performed on datasets!")    
#print(datasets)

The individual recipe files now contains information about how to preprocess(i.e. perform scaling) the datasets

In [13]:
#ID 12(3)


for (i,dataset_name) in enumerate(datasets):
    print(f"{i+1}.{new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml\n")
    !cat {new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml

1.Paper_A/20211223120704-usi/20211223120704-Paper_A-[myImageDataset]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
2.Paper_A/20211223120704-usi/20211223120704-Paper_A-[mnist]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098


In [14]:
#ID 13(3)                            ANOMALY-DATASET CONVERSION PERFORMED
 

datasets_ad={}    
    # for storing dataset objects converted to anomaly-dataset
for dataset_name in datasets:   
     datasets_ad[dataset_name]= UnsupervisedAnomalyDataset(classification_dataset=datasets[dataset_name],
                                                       normal_labels=0,
                                                       yamlpath_append=f"{new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml")   
     
        
                                                                            
print("datasets after adding anomaly-conversion datasets: ")    
print(datasets_ad)

datasets after adding anomaly-conversion datasets: 
{'myImageDataset': <oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7f90780e4810>, 'mnist': <oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7f8fbbf34e50>}


In [15]:
#ID 14(3)

for (i,dataset_name) in enumerate(datasets):
    print(f"{i+1}.{new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml\n")
    !cat {new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml

1.Paper_A/20211223120704-usi/20211223120704-Paper_A-[myImageDataset]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
2.Paper_A/20211223120704-usi/20211223120704-Paper_A-[mnist]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:


# **4. SAMPLING**

Here, we define the sampling parameters to sample from the datasets

In [16]:
#ID 15(4)


### ADD YOUR OWN SAMPLING PARAMETERS ###

# sampling parameters

n=25                              #Number of data points to sample                    
contamination_rate = 0.5         #Contamination rate when sampling, defaults to 0.05
n_steps = 1        # n_steps=10  #Number of samples to take, i.e., number of times sampling is repeated, defaults to 10   

#These below are the possible sampling types to sample from datasets
sampling_types=['unsupervised_multiple','unsupervised_single','unsupervised_multiple_benchmark']


sampling_type='unsupervised_multiple'  #by default for this run

sampling_params={'n':n,'contamination_rate':contamination_rate,'n_steps':n_steps,'flatten_images':False}



The above sampling parameters are utilized in
<b>#ID 23(5)</b> 
for sampling the datasets(except the pre-installed mv_tec_ad_datasets) before training the algorithms.

In [17]:
#ID 16(4)

benchmarking_datasets={}

for (x,y) in datasets_ad.items():
    benchmarking_datasets[x]=y

print(benchmarking_datasets)      

{'myImageDataset': <oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7f90780e4810>, 'mnist': <oab.data.unsupervised.UnsupervisedAnomalyDataset object at 0x7f8fbbf34e50>}


The above dictionary <b>benchmarking_datasets</b> will be used for the Benchmarking as it contains all the information:"
    
    
    1.dataset_name
    2.final_dataset_object(preprocessed and anomaly-converted)



# **5. ALGORITHM TRAINING AND TESTING**

<b>To load own algorithm(s), refer to #ID 30(9) and #ID 31(9)</b> where an example algorithm is loaded, then 

come back to this cell  and <b>load own algorithm(s) details in #ID 17(5),#ID 18(5) AND #ID 19(5)</b> in the same way as benchmark algorithms.

We first download and import algorithms used for anomaly decection.

In [18]:
#ID 17(5)

import wget

import wget

wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/cae_ABOD.py', "cae_ABOD.py")
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/cae_iforest.py', "cae_iforest.py")
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/conv_ae.py', "conv_ae.py")
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/cae_LOF.py', "cae_LOF.py")
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/cae_KNN.py', "cae_KNN.py")
### ADD your algo import(s) below ###


100% [............................................................] 1930 / 1930

'cae_KNN (2).py'

In [19]:
#ID 18(5)



### ADD your algo import(s) below ###

#from module_name import class_name

# load all algorithms
from conv_ae import ConvAutoEncoder
from cae_ABOD import CAEABOD
from cae_KNN import CAEKNN
from cae_LOF import CAELOF
from cae_iforest import CAEIForest


Firstly, we define hyperparameters for all algorithms and choose for benchmarking:

In [20]:
#ID 19(5)

   


### ADD YOUR OWN (HYPER)PARAMETERS AND THEIR VALUES FOR PRE-INSTALLED ALGOS###

# CAE+KNN
knn_factor = 0.05
knn_minimum = 10


# CAE+LOF
lof_factor = 0.1
lof_minimum = 10

# CAE+ABOD
abod_factor = 0.01
abod_minimum = 10



lst_benchmark_algorithms =[
   
      {   
       "algo_module_name": "cae_ABOD" , 
       "algo_class_name": "CAEABOD",
       "algo_name_in_result_table": "CAE v3",
       "algo_parameters": {"CAE_parameters": {'latent_dim': 100, 'epochs': 50, 'verbose': 0},
                           "ABOD_parameters": {'n_neighbors':{'abod_factor':0.1 ,'abod_minimum':10 }}}, 
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    } 
    
]
'''
 ### uncomment to use these algos  for benchmarking
 ,  
 
 {
       "algo_module_name": "conv_ae"   ,
       "algo_class_name": "ConvAutoEncoder",
       "algo_name_in_result_table": "CAE v1",
       "algo_parameters":   {'latent_dim': 100, 'epochs': 50, 'verbose': 0},
        "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
       } 
 ,
{   
       "algo_module_name": "cae_LOF" , 
       "algo_class_name": "CAELOF",
       "algo_name_in_result_table": "CAE v5",
       "algo_parameters": {"CAE_parameters": {'latent_dim': 100, 'epochs': 50, 'verbose': 0},
                           "LOF_parameters": {'n_neighbors': {'lof_factor':0.1 ,'lof_minimum':10 }}},
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    }

 
    ,

    {   
       "algo_module_name": "cae_KNN" , 
       "algo_class_name": "CAEKNN",
       "algo_name_in_result_table": "CAE v4",
       "algo_parameters": {"CAE_parameters": {'latent_dim': 100, 'epochs': 50, 'verbose': 0},
                           "KNN_parameters": {'n_neighbors': {'knn_factor':0.05 ,'knn_minimum':10 }}},
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_scores": {'field_name': 'decision_scores_'}
    },
     {
       "algo_module_name": "cae_iforest",   
       "algo_class_name": "CAEIForest",
       "algo_name_in_result_table": "CAE v2",
       "algo_parameters": {"CAE_parameters": {'latent_dim': 100, 'epochs': 50, 'verbose': 0}, "IForest_parameters": {'random_state': 42} },
        "fit": {'method_name': 'fit', 'params': {}}, 
        "decision_scores": {'field_name': 'decision_scores_'}
        }
    
    
]    
'''    

### ADD OWN ALGORITHMS NAME(S) with algorithm specifications as shown above for OAB algorithms ###   

own_algorithms=[]   #add to this list e.g. { "algo_module_name": "own_algo" , "algo_class_name": "ownAlgoClass",.........."decision_scores": 'decision_scores_'} 

lst_benchmark_algorithms.extend(own_algorithms)
recipe_datasets={}

#seed defined for ths benchmark run for obtaining consistent results 
seed=42


<b>LOAD YOUR RECIPE FOLDER CONTAINING ALL RECIPES</b> to be repdroduced and use it in the current benchmark run.

In [21]:
#ID 20(5)        # Execute this cell only when you already have a recipe folder  to load from 

### ADD AN OPTIONAL RECIPE FOLDER CONTAINING ALL DATASETS AND ALGOS RECIPES PATH TO ADD TO THIS BENCHMARK RUN START ###   

# Note: recipe folders of benchmarking type "unsupervised image(ssi) " i.e. of the format: 
#               "timestamp-usi"
# can only be used for benchmarking in this notebook.

recipes_folder= "Paper_B/20211222162118-usi" 

### ADD AN OPTIONAL RECIPE FOLDER CONATAINING ALL DATASETS AND ALGOS RECIPES PATH TO ADD TO THIS BENCHMARK RUN END ###   
 
    
    
### UNCOMMENT ONLY IF NO NEW DATASETS WERE ADDED IN THE BENCHMARK EXCEPT FROM RECIPE FOLDER START ###     

#benchmarking_datasets={}
#lst_benchmark_algorithms=[]

### UNCOMMENT ONLY IF NO NEW DATASETS WERE ADDED IN THE BENCHMARK EXCEPT FROM RECIPE FOLDER END ###  

for i,file in enumerate(os.listdir(recipes_folder)):
    print(f"{i+1}.{file}\n")
    !cat {recipes_folder}/{file}
    print("\n")

1.20211222162118-Paper_B-[myImageDataset2]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
seed: 42
sampling:
  unsupervised_multiple:
    n: 25
    n_steps: 1
    contamination_rate: 0.5
    shuffle: true
    random_seed: 42
    apply_random_seed: true
    keep_frequency_ratio_normals: false
    equal_frequency_normals: false
    keep_frequency_ratio_anomalies: false
    equal_frequency_anomalies: false
    include_description: true
    flatten_images: false


2.20211222162118-Paper_B-[mvtec_ad_transistor]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
seed: 42
sampling:
  unsupervised_multiple:
    n: 25
    n_steps: 1
    contamination_rate: 0.5
    shuffle: true
    random_seed: 42
    apply_random_seed: true
    keep_fre

In [22]:
#ID 21(5)    # Execute this cell only when you already have a recipe fOlder  to load from 

recipe_datasets,recipe_algos,seed=data_from_multiple_recipes(recipes_folder,new_recipe_path) # get information of all datasets and algos from the files in the recipes_folder 

# adding recipe_datasets to benchmarking_datasets
for dataset_name in recipe_datasets:
    benchmarking_datasets[dataset_name]=recipe_datasets[dataset_name][0]
#print(f"benchmarking_datasets: {benchmarking_datasets}") 
                                         
#adding algos from recipe_algos to lst_benchmarking_algos
for algo in recipe_algos:
    if algo not in lst_benchmark_algorithms:
     lst_benchmark_algorithms.append(algo)
    
#print(lst_benchmark_algorithms)   


myImageDataset2-----

preprocessing performed!!
anomaly-dataset transformation performed!!

mvtec_ad_transistor-----

preprocessing performed!!
anomaly-dataset transformation performed!!

cae_LOF----



In [23]:
#ID 22(5)

  
print("\nAll Datasets for this benchmark run:")    
for dataset_name in benchmarking_datasets:
    print(dataset_name)

    
 
print("\nAll algos for this benchmark run:")
for algo in lst_benchmark_algorithms:
    #print(algo)
    print(algo['algo_module_name'])




All Datasets for this benchmark run:
myImageDataset
mnist
myImageDataset2
mvtec_ad_transistor

All algos for this benchmark run:
cae_ABOD
cae_LOF


Now, For every benchmark dataset , we sample from that dataset to train the algorithms and then predict the outcomes for each dataset with each algortihm and then store results in a evaluation object, which is then added to the comparison object to show the final Benchmarking results

In [25]:
#ID 23(5)                # RUNNING THE BENCHMARK
co = ComparisonObject()
for dataset_name in benchmarking_datasets:
    print(f'-------{dataset_name}-------') 
    _append_to_yaml(f"{new_recipe_path}-[{dataset_name}]-dataset-recipe.yaml",'seed',seed)
    #print(mvtec_ad_own_datasets_list)
    for alg in lst_benchmark_algorithms:
        name=alg["algo_module_name"]
        c_name=alg["algo_class_name"]
        print(f"------{name}")
        eval_obj = EvaluationObject(algorithm_name=alg["algo_name_in_result_table"])
        
        if dataset_name in recipe_datasets:
            
          for (x,y), sample_config in benchmarking_datasets[dataset_name].sample_from_yaml(recipe_datasets[dataset_name][1]):
            print('.', end='') # update to see progress 
            w = UnsupervisedWrapperToRecipe()
            torch.manual_seed(seed)
            random.seed(seed)
            tf.random.set_seed(seed)
            np.random.seed(seed) 
            os.environ['PYTHONHASHSEED'] = str(seed)
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            torch.use_deterministic_algorithms(True)
            #print(alg["algo_parameters"])
            algo_parameters=algo_params(alg,l=len(x))
            #print(algo_parameters)
            algo= getattr(__import__(alg["algo_module_name"]),alg["algo_class_name"]) # Algo object imported from class 
            algo_initialized=algo(**algo_parameters)
            w.track_init(algo, params=alg["algo_parameters"])
            w.track_fit(x=x, obj=algo_initialized, params=alg['fit']['params'], fit_method=alg['fit']['method_name']) # the last parameter is the name of the method used for fitting
            pred = w.track_decision_scores(algo_initialized,field_name=alg['decision_scores']['field_name']) # the last parameter is the field name used to store anomaly scores by the model
            w.store_recipe(f"{new_recipe_path}-[{name}][{c_name}]-algo-recipe.yaml")
            _append_to_yaml(f"{new_recipe_path}-[{name}][{c_name}]-algo-recipe.yaml",'seed',seed)
            eval_obj.add(ground_truth=y, prediction=pred, description=sample_config) 
               
        else:
          
          for (x,y), sample_config in sample_v2(dataset_name,sampling_type,sampling_params,new_recipe_path,benchmarking_datasets[dataset_name]):
            print('.', end='') # update to see progress 
            w = UnsupervisedWrapperToRecipe()
            torch.manual_seed(seed)
            random.seed(seed)
            tf.random.set_seed(seed)
            np.random.seed(seed) 
            os.environ['PYTHONHASHSEED'] = str(seed)
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            torch.use_deterministic_algorithms(True)
            #print(alg["algo_parameters"])
            algo_parameters=algo_params(alg,l=len(x))
            
            #print(algo_parameters)
            algo= getattr(__import__(alg["algo_module_name"]),alg["algo_class_name"]) # Algo object imported from class 
            algo_initialized=algo(**algo_parameters)
            
            w.track_init(algo, params=alg["algo_parameters"])
            w.track_fit(x=x, obj=algo_initialized, params=alg['fit']['params'], fit_method=alg['fit']['method_name']) # the last parameter is the name of the method used for fitting
            pred = w.track_decision_scores(algo_initialized,field_name=alg['decision_scores']['field_name']) # the last parameter is the field name used to store anomaly scores by the model
            w.store_recipe(f"{new_recipe_path}-[{name}][{c_name}]-algo-recipe.yaml")
            _append_to_yaml(f"{new_recipe_path}-[{name}][{c_name}]-algo-recipe.yaml",'seed',seed)
            eval_obj.add(ground_truth=y, prediction=pred, description=sample_config) 
               
            
            
        eval_desc = eval_obj.evaluate(print=True,metrics=['roc_auc', 'adjusted_average_precision', 'precision_recall_auc'])
        co.add_evaluation(eval_desc)
        print("\n")
      

-------myImageDataset-------
------cae_ABOD
.Evaluation on dataset mvtec_ad_myImageDataset with normal labels [0] and anomaly labels [1.0].
Total of 1 datasets. Per dataset:
25 instances, contamination_rate 0.5.
Mean 	 Std_dev 	 Metric
0.282 	 0.000 		 roc_auc
-0.205 	 0.000 		 adjusted_average_precision
0.387 	 0.000 		 precision_recall_auc


------cae_LOF
.Evaluation on dataset mvtec_ad_myImageDataset with normal labels [0] and anomaly labels [1.0].
Total of 1 datasets. Per dataset:
25 instances, contamination_rate 0.5.
Mean 	 Std_dev 	 Metric
0.442 	 0.000 		 roc_auc
-0.029 	 0.000 		 adjusted_average_precision
0.463 	 0.000 		 precision_recall_auc


-------mnist-------
------cae_ABOD
.Evaluation on dataset mnist with normal labels [0] and anomaly labels [1, 2, 3, 4, 5, 6, 7, 8, 9].
Total of 1 datasets. Per dataset:
25 instances, contamination_rate 0.5.
Mean 	 Std_dev 	 Metric
0.673 	 0.000 		 roc_auc
0.329 	 0.000 		 adjusted_average_precision
0.661 	 0.000 		 precision_recall_auc


# **6. EVALUATION**

Here , we will see how different metrics can be selected when evaluating an algorithm's performance.

In previous section while creating an evalutation description,  we used all metrics for evaluation:

     eval_desc = eval_obj.evaluate(print=False, metrics=all_metrics)
    
    

In [26]:
#ID 24(6)

# to use a subset, first see which ones are available

print(all_metrics)

['roc_auc', 'average_precision', 'adjusted_average_precision', 'precision_n', 'adjusted_precision_n', 'precision_recall_auc']


In [27]:
#ID 25(6)

#### ADD YOUR OWN NUMBER OF METRICS ###

#Then we can  select an arbitrary subset
metrics=['roc_auc', 'precision_recall_auc']

# **7. SHOW BENCHMARK RESULTS**

We compare by printing, the results of the evaluations of different Algo-Dataset combinations.

\[Latex version: bold for highest, italics for second highest, ?\]

In [28]:
#ID 26(7)

# print results in easily readable format
co.print_results()

For roc_auc:
         mvtec_ad_myImageDataset     mnist  mvtec_ad_myImageDataset2  \
CAE v3                  0.282051  0.673077                  0.403846   
cae_LOF                 0.442308  0.525641                  0.397436   
Average                 0.362179  0.599359                  0.400641   

         mvtec_ad_transistor   Average  
CAE v3              0.532051  0.472756  
cae_LOF             0.519231  0.471154  
Average             0.525641       NaN  
For adjusted_average_precision:
         mvtec_ad_myImageDataset     mnist  mvtec_ad_myImageDataset2  \
CAE v3                 -0.204803  0.328967                 -0.075947   
cae_LOF                -0.029050  0.130423                 -0.073227   
Average                -0.116927  0.229695                 -0.074587   

         mvtec_ad_transistor   Average  
CAE v3              0.302684  0.087725  
cae_LOF             0.278510  0.076664  
Average             0.290597       NaN  
For precision_recall_auc:
         mvtec_ad_myIma

In [29]:
#ID 27(7)

# print results in easily readable format with standard deviations
co.print_results(include_stdevs=True)

For roc_auc:
        mvtec_ad_myImageDataset         mnist mvtec_ad_myImageDataset2  \
CAE v3             0.282+-0.000  0.673+-0.000             0.404+-0.000   
cae_LOF            0.442+-0.000  0.526+-0.000             0.397+-0.000   
Average                   0.362         0.599                    0.401   

        mvtec_ad_transistor   Average  
CAE v3         0.532+-0.000  0.472756  
cae_LOF        0.519+-0.000  0.471154  
Average               0.526       NaN  

For adjusted_average_precision:
        mvtec_ad_myImageDataset         mnist mvtec_ad_myImageDataset2  \
CAE v3            -0.205+-0.000  0.329+-0.000            -0.076+-0.000   
cae_LOF           -0.029+-0.000  0.130+-0.000            -0.073+-0.000   
Average                  -0.117         0.230                   -0.075   

        mvtec_ad_transistor   Average  
CAE v3         0.303+-0.000  0.087725  
cae_LOF        0.279+-0.000  0.076664  
Average               0.291       NaN  

For precision_recall_auc:
        mvtec

In [30]:
# ID 28(7)

co.print_latex(include_stdevs=True)

For roc_auc:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & mvtec\_ad\_myImageDataset2 & mvtec\_ad\_transistor & Average \\
  CAE v3 & \textit{0.282$\pm$0.000} & \textbf{0.673$\pm$0.000} & \textbf{0.404$\pm$0.000} & \textbf{0.532$\pm$0.000} & \textbf{0.473} \\
  cae\_LOF & \textbf{0.442$\pm$0.000} & \textit{0.526$\pm$0.000} & \textit{0.397$\pm$0.000} & \textit{0.519$\pm$0.000} & \textit{0.471} \\
  Average & 0.362 & 0.599 & 0.401 & 0.526 &    \\
\end{tabular}
\end{center}

For adjusted_average_precision:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & mvtec\_ad\_myImageDataset2 & mvtec\_ad\_transistor & Average \\
  CAE v3 & \textit{-0.205$\pm$0.000} & \textbf{0.329$\pm$0.000} & \textit{-0.076$\pm$0.000} & \textbf{0.303$\pm$0.000} & \textbf{0.088} \\
  cae\_LOF & \textbf{-0.029$\pm$0.000} & \textit{0.130$\pm$0.000} & \textbf{-0.073$\pm$0.000} & \textit{0.279$\pm$0.000} & \textit{0.077} \\
  Average & -0.11

# **8. REPRODUCIBILITY**

 ## **8.1 Creating recipes**

This section shows **how `oab` can be used to make sampling results easily reproducible** .
 

`yaml` files play an integral role in making reproducibility work, as they store the operations and parameters performed on the data.

We will see how to produce a recipe(.yaml) of the Benchmarkrun already performed  in <b>#ID 23(5)</b>

In <b>#ID 11(3) #ID 13(3) and #ID 23(5) </b>,  We already performed `creating_recipe` operations on </b>own datasets,OAB's datasets as well as input recipe's datasets and all algorithms of this benchmark run,we can see below  the structure of those recipes:


In [31]:
#ID 29(8)


for i,file in enumerate(os.listdir(f"{benchmark_name}/{time}-{benchmark_type}")):
    print(f"{i+1}.{file}\n")
    !cat {benchmark_name}/{time}-{benchmark_type}/{file}
    print("\n")

1.20211223120704-Paper_A-[myImageDataset]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
seed: 42
sampling:
  unsupervised_multiple:
    n: 25
    n_steps: 1
    contamination_rate: 0.5
    shuffle: true
    random_seed: 42
    apply_random_seed: true
    keep_frequency_ratio_normals: false
    equal_frequency_normals: false
    keep_frequency_ratio_anomalies: false
    equal_frequency_anomalies: false
    include_description: true
    flatten_images: false


2.20211223120704-Paper_A-[myImageDataset2]-dataset-recipe.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
seed: 42
sampling:
  unsupervised_multiple:
    n: 25
    n_steps: 1
    contamination_rate: 0.5
    shuffle: true
    random_seed: 42
    apply_random_seed: true
    keep_frequenc

### 2. Reproducing the experiment

To reproduce the recipes created in the previous section,
we refer to <b>Section 5 #ID 20(5)</b> where we can reproduce the run as well as extend benchmarks!

# **9. EXTEND EXISTING BENCHMARK(own algorithm)**

To extend the existing benchmark here basically means to add  our own algorithm to the benchmark and to show the comparison results of pre-installed algorithms while also loading our own dataset.


1. We load the datasets. To know how to do that, we can refer to  **Section "1. Data" and "2. Data Selection"**
2. Then, load own algorithm as we will see in the next sub-section.

## **9.1 Loading own Algorithm**

In this subsection, you will see **how an own unsupervised anomaly detection algorithm** can easily be used within oab to be evaluated. We will see how a class representing an algorithm can be structured and how its performance is evaluated.

Of course, this is not the only way to use the functionality provided by oab. We do consider it to be the simplest way however.

In [32]:
#ID 30(9)

# download example algorithm and inspect content
import wget
wget.download('https://raw.githubusercontent.com/jandeller/test/main/RandomGuesser.py',"RandomGuesser.py")
!cat RandomGuesser.py

100% [..............................................................] 204 / 204import numpy as np

class RandomGuesser():

    def fit(self, X):
        "Assign a random number to each sample"
        n_samples = X.shape[0]
        self.decision_scores_ = np.random.randn(n_samples)


The sample `RandomGuesser` algorithm shown here is - as the name suggests - a random guesser, i.e., it assigns random anomaly scores to the samples.

An algorithm used for unsupervised anomaly detection needs to specify a `fit(X_train)` method for training and a `decision_scores_` variable for inference that returns an anomaly score per data point in the test set.

It is of course possible to rename the method and field, use a method for accessing the anomaly scores, etc. Note that if this is done, the following code has to be changed accordingly. Adhering to the conventions described above (`fit(X_train)` and `decision_scores_`) allows you to use the same interface as algorithms from [`PyOD`](https://pyod.readthedocs.io/en/latest/) as shown when [comparing algorithms using `oab`](https://colab.research.google.com/drive/1aV_itaYCJgzdZ1lQ7SUyHQ7z01xSPxDN?usp=sharing#scrollTo=QnAfCGTGL7xv).

In [33]:
#ID 31(9)
# used imports from #ID 3(0),#ID 18(5)
#used sampling parameters from #ID 14(4)

# and import the RandomGuesser
from RandomGuesser import RandomGuesser

own_algorithms=[{
    
       ### ADD YOUR OWN ALGO DETAILS IN THIS FORM ###
       "algo_module_name": "RandomGuesserSemisupervised",   
       "algo_class_name": "RandomGuesserSemisupervised",
       "algo_name_in_result_table": "RandomGuesserSemisupervised",
       "algo_parameters": {},
        "fit": {'method_name': 'fit', 'params': {}}, 
        "decision_SCORES": {'field_name': 'decision_scores_'}
        }]


The `own_algorithms` list in the above cell #ID 31(9) can be added to `lst_benchmarking_algos` as mentioned in #ID 19(5) to use this algorithm in a benchmark run shown in #ID 23(5) along with other algorithms

In [34]:
#ID 35(9)

#  A comparison object is created for comparing the evaluations of different Algo-Dataset combinations
co = ComparisonObject()

for dataset_name in benchmarking_datasets:
  # evaluate the random guesser
  eval_obj = EvaluationObject(algorithm_name="RandomGuesser")
  for (x,y), settings in benchmarking_datasets[dataset_name].sample_multiple(n=n, 
                                                                  contamination_rate=contamination_rate, 
                                                                 n_steps=n_steps):
      print(".", end=" ") # update to see progress
      rg = RandomGuesser()
      rg.fit(x) # data is fitted to RandomGuesser
      pred = rg.decision_scores_ # and decision_scores_ is accessed
      eval_obj.add(y, pred, settings)
  print("\n")
  eval_desc = eval_obj.evaluate(metrics=['roc_auc', 'adjusted_average_precision', 'precision_recall_auc'])
  # added to comparison object
  co.add_evaluation(eval_desc)
  print("\n")



. 

Evaluation on dataset mvtec_ad_myImageDataset with normal labels [0] and anomaly labels [1.0].
Total of 1 datasets. Per dataset:
25 instances, contamination_rate 0.5.
Mean 	 Std_dev 	 Metric
0.455 	 0.000 		 roc_auc
-0.006 	 0.000 		 adjusted_average_precision
0.474 	 0.000 		 precision_recall_auc


. 

Evaluation on dataset mnist with normal labels [0] and anomaly labels [1, 2, 3, 4, 5, 6, 7, 8, 9].
Total of 1 datasets. Per dataset:
25 instances, contamination_rate 0.5.
Mean 	 Std_dev 	 Metric
0.571 	 0.000 		 roc_auc
0.085 	 0.000 		 adjusted_average_precision
0.524 	 0.000 		 precision_recall_auc


. 

Evaluation on dataset mvtec_ad_myImageDataset2 with normal labels [0] and anomaly labels [1.0].
Total of 1 datasets. Per dataset:
25 instances, contamination_rate 0.5.
Mean 	 Std_dev 	 Metric
0.564 	 0.000 		 roc_auc
0.197 	 0.000 		 adjusted_average_precision
0.590 	 0.000 		 precision_recall_auc


. 

Evaluation on dataset mvtec_ad_transistor with normal labels [0] and anomaly l

As in the above code, We store the evaluations of our own algorithm in evaluation object which is then added to comparison object.Similarly, we can create evaluation objects for other algorithms and add them to comparison object for final benchmarking  as shown in Section 5

Finally, we show below the benchmarking results of our algorithm as described in "**Section 7. Show Benchmarking Results**"

In [35]:
#ID 33(9)

# print results in easily readable format
co.print_results()

For roc_auc:
               mvtec_ad_myImageDataset     mnist  mvtec_ad_myImageDataset2  \
CAE v3                        0.282051  0.673077                  0.403846   
cae_LOF                       0.442308  0.525641                  0.397436   
RandomGuesser                 0.455128  0.570513                  0.564103   
Average                       0.393162  0.589744                  0.455128   

               mvtec_ad_transistor   Average  
CAE v3                    0.532051  0.472756  
cae_LOF                   0.519231  0.471154  
RandomGuesser             0.487179  0.519231  
Average                   0.512821       NaN  
For adjusted_average_precision:
               mvtec_ad_myImageDataset     mnist  mvtec_ad_myImageDataset2  \
CAE v3                       -0.204803  0.328967                 -0.075947   
cae_LOF                      -0.029050  0.130423                 -0.073227   
RandomGuesser                -0.006266  0.085048                  0.196601   
Average          

In [37]:
#ID 34(9)
# print results in easily readable format with standard deviations
co.print_results(include_stdevs=True)

For roc_auc:
              mvtec_ad_myImageDataset         mnist mvtec_ad_myImageDataset2  \
CAE v3                   0.282+-0.000  0.673+-0.000             0.404+-0.000   
cae_LOF                  0.442+-0.000  0.526+-0.000             0.397+-0.000   
RandomGuesser            0.455+-0.000  0.571+-0.000             0.564+-0.000   
Average                         0.393         0.590                    0.455   

              mvtec_ad_transistor   Average  
CAE v3               0.532+-0.000  0.472756  
cae_LOF              0.519+-0.000  0.471154  
RandomGuesser        0.487+-0.000  0.519231  
Average                     0.513       NaN  

For adjusted_average_precision:
              mvtec_ad_myImageDataset         mnist mvtec_ad_myImageDataset2  \
CAE v3                  -0.205+-0.000  0.329+-0.000            -0.076+-0.000   
cae_LOF                 -0.029+-0.000  0.130+-0.000            -0.073+-0.000   
RandomGuesser           -0.006+-0.000  0.085+-0.000             0.197+-0.000   
Ave

In [36]:
#ID 35(9)

co.print_latex(include_stdevs=True)

For roc_auc:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & mvtec\_ad\_myImageDataset2 & mvtec\_ad\_transistor & Average \\
  CAE v3 & 0.282$\pm$0.000 & \textbf{0.673$\pm$0.000} & \textit{0.404$\pm$0.000} & \textbf{0.532$\pm$0.000} & \textit{0.473} \\
  cae\_LOF & \textit{0.442$\pm$0.000} & 0.526$\pm$0.000 & 0.397$\pm$0.000 & \textit{0.519$\pm$0.000} & 0.471 \\
  RandomGuesser & \textbf{0.455$\pm$0.000} & \textit{0.571$\pm$0.000} & \textbf{0.564$\pm$0.000} & 0.487$\pm$0.000 & \textbf{0.519} \\
  Average & 0.393 & 0.590 & 0.455 & 0.513 &    \\
\end{tabular}
\end{center}

For adjusted_average_precision:
\begin{center}
\begin{tabular}{  c c c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & mvtec\_ad\_myImageDataset2 & mvtec\_ad\_transistor & Average \\
  CAE v3 & -0.205$\pm$0.000 & \textbf{0.329$\pm$0.000} & -0.076$\pm$0.000 & \textbf{0.303$\pm$0.000} & \textit{0.088} \\
  cae\_LOF & \textit{-0.029$\pm$0.000} & \textit{0.130$\pm$0.000} & \textit{-

So,This was our example algorithm. Other algorithms can be used to run and extend benchmarks,  Please refer  to <b>5. ALGORITHM TRAINING AND TESTING</b>.